# Getting started

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import ompy as om
import logging

In [3]:
om.__full_version__

'1.1.0.dev0+c133e43'

In [4]:
# For reproducability we seed the random generator. 
# Note that by default several other classes in ompy, such as all
# classes with multinest calculations have a default seed, too
np.random.seed(1382398)

In [5]:
# get smaller files for the online version
plt.rcParams["figure.dpi"] = 70

## Loading and example raw spectra

The $^{164}\mathrm{Dy}$ data used below has been gathered from following experiment: Nyhus, H. T. *et al.* (2010). DOI: [10.1103/physrevc.81.024325](https://doi.org/10.1103/PhysRevC.81.024325)
and is reanalyzed in Renstrøm, T. *et al.* (2018). DOI: [10.1103/physrevc.98.054310](https://doi.org/10.1103/PhysRevC.98.054310)

In [6]:
# Import raw matrix into instance of om.Matrix() and plot it
raw = om.example_raw('Dy164')
# To use you own data, uncomment/adapt the line below instead
# raw = om.Matrix(path="/path/to/matrix.ending")

# Plot the entire matrix
raw_org = raw.copy() # workaround due to execution order in jupyter notebook
                     # (calculations are performed before plotting, but we make a cut to raw further down)
raw_org.plot();


# Note: We use the semi-colon `;` at the end of the line to silence the output
# in jupyter notebook. This is not necessary, but otherwise you get something like 
# this below printed every time:
#(<matplotlib.collections.QuadMesh at 0x7fafbc422eb8>,
# <matplotlib.axes._subplots.AxesSubplot at 0x7fafc0944a20>,
# <Figure size 640x480 with 2 Axes>)

<IPython.core.display.Javascript object>

## Matrix manipulation

The core of the Oslo method involves working with two dimensional spectra. Starting with a raw matrix of $E_x$-$E_\gamma$ coincidences, you typically want to unfold the counts
along the gamma-energy axis and then apply the first-generation method to obtain the matrix of first-generation, or primary, gamma rays from the decaying nucleus.

The two most important utility classes in the package are `Matrix()` and `Vector()`. They are used to store matrices (2D) or vectors (1D) of numbers, typically spectra of counts, along with energy calibration information. 

As these underpin the entire package, they contain many useful functions to make life easier. Loading and saving to several formats, plotting, projections, rebinning and cutting, to mention a few. See the documentation for an exhaustive list.

Their basic structure is:

In [7]:
# mat = ompy.Matrix()
mat = raw
mat.values  # A 2D numpy array
mat.Ex      # Array of mid-bin energy values for axis 0 (i.e. the row axis, or y axis)
mat.Eg      # Array of mid-bin energy values for axis 1 (i.e. the column axis, or x axis)

print("The first gamma-ray energies:\n", mat.Eg[0:10])

The first gamma-ray energies:
 [  0.     19.364  38.728  58.092  77.456  96.82  116.184 135.548 154.912
 174.276]


In [8]:
# We can also create a vector, which is useful to store the NLD and gSF.
values = np.arange(11)
E = np.linspace(0, 10, num=11)

fig, ax = plt.subplots(figsize=(2,2), constrained_layout=True)
vec = om.Vector(values=values, E=E)
vec.values  # A 1D numpy array
vec.E       # Array of lower-bin-edge energy values for the single axis
vec.plot(ax=ax);

<IPython.core.display.Javascript object>

In [9]:
# Cut away counts above the diagonal 
# Remember: Think about what you do here. If you cut them away, they will not
# be used in unfolding etc. This may or may not be what you want.
# Note that the raw matrix we read in above has been cut already, so the difference here is not so large.
raw.cut_diagonal(E1=(800, 0), E2=(7500, 7300))
raw.cut('Ex', 0, 8400)
raw.plot();

<IPython.core.display.Javascript object>

Note that `Matrix`, `Vector` and several other classes contain mutable objects. If you work on them, you might want to create a *deepcopy*. For `Matrix`, `Vector` this can be archived by the convince method `X.copy`, otherwise use `copy.deepcopy`.

In [10]:
# The "right" way if you don't want to change the original matrix
raw_big_cut = raw.copy()
raw_big_cut.cut('Ex', 0, 4000)
print(raw.Ex.max(), raw_big_cut.Ex.max())

8300.0 3980.0


In [11]:
# The "wrong" way if you don't want to change the original matrix
raw_big_cut2 = raw_big_cut
raw_big_cut2.cut('Ex', 0, 2000)
print(raw_big_cut.Ex.max(), raw_big_cut2.Ex.max())
# oups!: suddenly also `raw_big_cut` was cut, not only raw_big_cut2

1940.0 1940.0


In [12]:
# Plot projections
raw.plot_projection('Ex', Emin=1800, Emax=2600, kind="step");

<IPython.core.display.Javascript object>

Note that you can IPython's has tools to quickly access information on a function, namely the `?` character to explore documentation, the `??` characters to explore source code, and the `Tab key` (or `double-tab`) for auto-completion. Try it out uncommenting the function below.

In [13]:
## Uncomment these lines to query a function
# ?raw.plot_projection

## Unfolding

### Get a response matrix
Eventhough this experiment was performed with CACTUS, we will now give an example *as if* the data was taken with OSCAR. This is beacause it is slightly easier to handle. For CACTUS, the response functions do not include the detector threshold; this has do be taken into account separately.

In [14]:
logger = om.introspection.get_logger('response', 'INFO')
# Then do the same using OMpy functionality:
# You may need to adpot this to whereever you response matrices are stored
folderpath = "../OCL_response_functions/oscar2017_scale1.15"

# Energy calibration of resulting response matrix:
Eg = raw.Eg

# Experimental relative FWHM at 1.33 MeV of resulting array
fwhm_abs = 30 # (30/1330 = 2.25% )

# Magne recommends 1/10 of the actual resolution for unfolding purposes
response = om.Response(folderpath)
R_ompy_view, R_tab_view = response.interpolate(Eg, fwhm_abs=fwhm_abs, return_table=True)
R_ompy_unf, R_tab_unf = response.interpolate(Eg, fwhm_abs=fwhm_abs/10, return_table=True)
R_ompy_view.plot(title="Response matrix", vmin=5e-5, vmax=5e-1,
                 scale="log");



2020-08-14 15:47:56,002 - ompy.response - INFO - Note: Spectra outside of 200.0 and 20000.0 are extrapolation only.
2020-08-14 15:48:03,029 - ompy.response - INFO - Note: Spectra outside of 200.0 and 20000.0 are extrapolation only.


<IPython.core.display.Javascript object>

### Perform the unfolding

In [15]:
# You can decide to log information and set the logging level (info/debug)
logger = om.introspection.get_logger('unfolder', 'INFO')

# We need to remove negative counts (unphysical) in the raw matrix before unfolding:
raw_positive = raw.copy()
raw_positive.fill_and_remove_negative(window_size=2)

# With compton subtraction and all tweaks
unfolder= om.Unfolder(response=R_ompy_unf)
unfolder.use_compton_subtraction = True # default
unfolder.response_tab = R_tab_unf
# Magne suggests some "tweaks" for a better unfolding performance. Default is 1 for all.
unfolder.FWHM_tweak_multiplier = {"fe": 1., "se": 1.1,
                                     "de": 1.3, "511": 0.9}
unfolded = unfolder(raw_positive)
unfolded.plot();

<IPython.core.display.Javascript object>

### Generate the first generation matrix

In [16]:
firstgen = om.FirstGeneration()
primary = firstgen(unfolded)
primary.plot();

<IPython.core.display.Javascript object>

## Propagating statistical uncertainties

In order to propagate the statistical uncertainties from the raw matrix, we use an ensemble based method. We start of my generating en ensemble of *raw-like* matrices. The raw counts are Poisson distributed. If we had counted one another time, we would get slightly different results. 

More precisely, the counts of the matrix containing `prompt+bg` events and the background events `bg` are each Poisson distributed, where we have `raw = (prompt+bg) - bg_ratio * bg`. The ratio `bg_ratio` is determined by the ratio of the time gate lengths taken to obtain the `prompt+bg` and `bg` spectra. If a `bg` spectrum is provided to the `Ensemble` class, it will calculate the raw spectrum according to the equation above. Otherwise, the provided `raw` spectrum itself is assumed to be Poisson distributed.

We take the number of counts $k_i$ in bin $i$ of the raw matrix $R$ as an estimate for the Poisson parameter ("the mean") $λ_i$ . Note that it is an unbiased estimator for $λ_i$, since $E(k) = λ$. To generate a member matrix $R_l$ of the MC ensemble, we replace the counts in each bin $i$ by a random draw from the distribution $\operatorname{Poisson}(k_i)$.

The class Ensemble() provides this feature. Its basic usage is:

In [17]:
logger = om.introspection.get_logger('ensemble', 'INFO')

# Tell the `Ensemble` class which raw spectrum, what kind of undolfer and first
# generations method to use.
# Note: This will have the same setting as above. We could for example have
# set the first generations method to use a different "valley_collection", or a
# differnt type of "multiplicity_estimation"
ensemble = om.Ensemble(raw=raw_positive)
ensemble.unfolder = unfolder
ensemble.first_generation_method = firstgen
# Generates N perturbated members; here just 10 to speed it up
# the `regernerate` flag ensures, that we don't load from disk; which might result in expected results
# if we have changed something in the input `raw` matrix.
ensemble.generate(10, regenerate=True)

2020-08-14 15:48:13,148 - ompy.ensemble - INFO - Start normalization with 7 cpus
2020-08-14 15:48:13,210 - ompy.ensemble - INFO - Generating/loading 0
2020-08-14 15:48:13,232 - ompy.ensemble - INFO - Generating/loading 1
2020-08-14 15:48:13,263 - ompy.ensemble - INFO - Generating/loading 2
2020-08-14 15:48:13,293 - ompy.ensemble - INFO - Generating/loading 3
2020-08-14 15:48:13,323 - ompy.ensemble - INFO - Generating/loading 4
2020-08-14 15:48:13,360 - ompy.ensemble - INFO - Generating/loading 5
2020-08-14 15:48:13,397 - ompy.ensemble - INFO - Generating/loading 6


2020-08-14 15:48:18,082 - ompy.ensemble - INFO - Generating/loading 7
2020-08-14 15:48:18,144 - ompy.ensemble - INFO - Generating/loading 8
2020-08-14 15:48:18,204 - ompy.ensemble - INFO - Generating/loading 9



The generated members are saved to disk and can be retrieved. Unfolded members can be retrieved as `ensemble.get_unfolded(i)`, for example. Their standard deviation is `ensemble.std_unfolded` for the unfolded matrices, etc.

We can now plot the standard deviation of all ensemble members for the raw, unfolded and first generation spectrum 

In [18]:
i_unfolded = 9
matrix = ensemble.get_unfolded(i_unfolded)
matrix.plot(title=f"Unfolded matrix #{i_unfolded}")

# Following commands plots all std. deviations
ensemble.plot();

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Extract Nuclear level density and gamma strength function

After matrix has been cut, unfolded and firstgen'd, perhaps ensembled, its nuclear level density (nld) and gamma strength function ($\gamma$SF) can be extracted using the `Extractor()` class.  

The method relies on the relation
 \begin{align}
	P(E_x, E_\gamma) \propto NLD(E_x - E_\gamma) \mathcal{T}(E_\gamma),\label{eq:Oslo_method_eq}
\end{align}
where $P(E_x, E_\gamma)$ is the first-generation spectrum normalized to unity for each $E_x$ bin.  
Furthermore, if we assume that the $\gamma$ decay at high $E_x$ is dominated by dipole radiation the transmission coefficient $\mathcal{T}$ is related to the dipole $\gamma$-ray strength function $f(E_\gamma)$ by the relation
\begin{align}
    \mathcal{T}(E_\gamma) = 2\pi E_\gamma^3 f(E_\gamma).\label{eq:gammaSF}
\end{align} 

If you have reasons to assume a different multipole decomposition, you may of course calculate the transmission coefficient $\mathcal{T}$ from the $\gamma$-ray strength function produced here and apply the decomposition you prefer.

For a single matrix, its usage is:  
(well, think about what you want to set in as the std. deviation)

In [19]:
# cutout = primary.trapezoid(Ex_min=4000, Ex_max=8000, Eg_min=1000, inplace=False)
# cutout_std = ensemble.std_firstgen.trapezoid(Ex_min=4000, Ex_max=8000, Eg_min=1000, inplace=False)
# extractor = om.Extractor()
# nld, gsf = extractor.decompose(cutout, std=cutout_std)

When extracting NLD and GSF from an ensemble, a trapezoidal cutout must be performed on each ensemble member. This is achieved by `Action()` which allows for delayed function calls on matrices and vectors. This way we don't cut the raw matrix at `Ex_min`, but this will only happen before the extraction.

In [20]:
# logger = om.introspection.get_logger('extractor', 'INFO')
trapezoid_cut = om.Action('matrix')
trapezoid_cut.trapezoid(Ex_min=4000, Ex_max=7000, Eg_min=1000, Eg_max=7000+200, inplace=True)
extractor = om.Extractor()

# Running the lines below directy, would most probably 
# result in a error like
# The AssertionError: Ex and Eg must have the same step size
#
# Why? The extraction assumes that Ex and Eg have the same binning. Thus we
# need to rebin the ensemble. This works will work inplace. 
# Note: As always, be careful will mid-bin vs lower bin calibration.
# E_rebinned = ensemble.get_firstgen(0).Ex
# 
E_rebinned = np.arange(100., 8500, 200)
ensemble.rebin(E_rebinned, member="firstgen")
ensemble.plot();

<IPython.core.display.Javascript object>

Now we can extract the NLD and $\gamma SF$ for $N$ of the samples of the ensemble.

Note: 
The old software extended the decomposition beyond the `Ex=Eg` line by a resolution `dE`. This is now optional and we changed the default to not do this any longer, but rather assume that the rebinning above has been performed with a binsize of approx. the FWHM of the bin with the worst resolution (usually `(Ex_max, Eg_max)`).

In [21]:
extractor.extract_from(ensemble, trapezoid_cut, regenerate=True)

The resulting `nld` and `gsf` are saved to disk and exposed as `extractor.nld` and `extractor.gsf`

In [22]:
mat = ensemble.get_firstgen(0).copy()
std = ensemble.std_firstgen.copy()
trapezoid_cut.act_on(mat)
trapezoid_cut.act_on(std)
_, _, product = extractor.decompose(mat, std, product=True)
fig, ax = plt.subplots(2,1)
om.normalize_rows(mat.values) 
mat.plot(ax=ax[0], scale="log", vmin=1e-3, vmax=1e-1)
product.plot(ax=ax[1], scale="log", vmin=1e-3, vmax=1e-1)

x = np.linspace(*ax[0].get_ylim())
ax[0].plot(x, x, "r--", label="E_x = E_g")
ax[1].plot(x, x, "r--", label="E_x = E_g")

<IPython.core.display.Javascript object>

In [23]:
nld = extractor.nld[0].copy()
gsf = extractor.gsf[0].copy()
resolution = np.zeros_like(mat.Ex)
E_nld = nld.E
Eg = mat.Eg
Ex = mat.Ex

# nld.values /= Eg
gsf.values *= Eg


T = (2*np.pi*Eg**3) * gsf.values
values = om.nld_T_product(nld.values, T, resolution,
                          E_nld, Eg, Ex)

mat = product.copy()
mat.values = values
mat.plot(scale="log", vmin=1e-3, vmax=1e-1)

<IPython.core.display.Javascript object>

(<matplotlib.collections.QuadMesh at 0x1257f71f0>,
 <AxesSubplot:xlabel='$\\gamma$-ray energy $E_{\\gamma}$', ylabel='Excitation energy $E_{x}$'>,
 <Figure size 448x336 with 2 Axes>)

### Plotting the results before normalization

In [24]:
extractor.plot(plot_mean=False)

<IPython.core.display.Javascript object>

(<Figure size 448x336 with 2 Axes>,
 array([<AxesSubplot:title={'center':'Level density'}>,
        <AxesSubplot:title={'center':'$\\gamma$SF'}>], dtype=object))

Or maybe you are more used to displaying the results with std. deviations?

**Note**: This may be erroneous, as the nld and gsf are not normalized yet!  
Thus, in principal, we might evaluate std. devs. of the *same solution* with different  
transformations. Before we normalize, we don't know. And they have the same $\chi^2$.  
That was the reason for the *trouble* with normalization.



In [25]:
extractor.plot(plot_mean=True)

<IPython.core.display.Javascript object>

(<Figure size 448x336 with 2 Axes>,
 array([<AxesSubplot:title={'center':'Level density'}>,
        <AxesSubplot:title={'center':'$\\gamma$SF'}>], dtype=object))

In [26]:
# let's remove the nan-valued elements (unconstrained elements) for the further analysis
for nld in extractor.nld:
    nld.cut_nan()

for gsf in extractor.gsf:
    gsf.cut_nan()

# the "mean" nld at this stage; we'll use it later, but it's not a good estimate at this
# stage (see article)
nld_mean = extractor.ensemble_nld()

## Normalization

Does it still look *strange*? probably because you are only used to see the normalized results.

### 1) Manual normalization

In [27]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def plot_transformed(alpha, A=1, B=1):
    fig, ax = plt.subplots(1, 2, constrained_layout=True)
    for nld, gsf in zip(extractor.nld, extractor.gsf):
        nld.transform(const=A, alpha=alpha, inplace=False).plot(ax=ax[0], scale="log", color='k', alpha=1/10)
        gsf.transform(const=B, alpha=alpha, inplace=False).plot(ax=ax[1], scale="log", color='k', alpha=1/10)
    ax[0].set_title("Level density")
    ax[1].set_title("γSF")

plot_transformed(alpha=0.0015)

<IPython.core.display.Javascript object>

### 2) Normalization through external data for one (nld, gsf) set

The normalization ensures that we find the *physical* solution, so we remove the degeneracy that is in principal inherent to decomposition of NLD and $\gamma$SF:
\begin{align}
NLD' = NLD(E_x) * A exp(\alpha E_x) \\
\gamma SF' = \gamma SF(E_\gamma) * B exp(\alpha E_\gamma)
\end{align}
Note: This is the transformation eq (3), Schiller2000.

As external data for the normalization we commonly use:
1. the discrete levels, binned with the resolution of our data (and potentially also smoothed)
2. The NLD at Sn, derived from D0 and a spin distribution
3. The average total radiative width $\Gamma_\gamma$.

#### 1. Sequentially:
Traditionally we have chosen a sequential normalization, where the NLD is normalized first to receive a set $\alpha$. Then we obtain the scaling parameter $B$ of the $\gamma$SF from a normalization to the experimental $\Gamma_\gamma$.

##### nld normalization  
Let's first normalize the mean nld from the extractor.

The normalization will take some time (≲ 30 seconds). The essential output of MultiNest is saved to disk, and some output is redirected to disk.

In [28]:
normlog = om.introspection.get_logger('normalizer_nld', 'INFO')
nldnorm = om.NormalizerNLD(nld=nld_mean, discrete='../example_data/discrete_levels_Dy164.txt',
                           regenerate=True)

In [29]:
norm_pars = om.NormalizationParameters(name="164Dy")
norm_pars.D0 = [6.8, 0.6]  # eV
norm_pars.Sn = [7.658, 0.001] # MeV
norm_pars.spincutModel = 'Disc_and_EB05'  # see eg. Guttormsen et al., 2017, PRC 96, 024313
norm_pars.spincutPars = {"mass":164, "NLDa":18.12, "Eshift":0.31,
                         "Sn": norm_pars.Sn[0], "sigma2_disc":[1.5, 3.6]}
norm_pars.Jtarget = 5/2 # A-1 nucleus

nldnorm.normalize(limit_low=[0, 1.5], limit_high=[3, 5.5], norm_pars=norm_pars)

2020-08-14 15:48:23,602 - ompy.normalizer_nld - INFO - 

---------
Normalizing nld #0
2020-08-14 15:48:24,910 - ompy.normalizer_nld - INFO - DE results:
┌───────────────────┬────────────────────┬────────────────────┬──────────────────────┐
│ A                 │ α [MeV⁻¹]          │ T [MeV]            │ Eshift [MeV]         │
╞═══════════════════╪════════════════════╪════════════════════╪══════════════════════╡
│ 1.543766998776277 │ 1.3373740052230374 │ 0.5622894816742635 │ -0.30956783037913443 │
└───────────────────┴────────────────────┴────────────────────┴──────────────────────┘
2020-08-14 15:48:24,911 - ompy.normalizer_nld - INFO - Starting multinest
  analysing data from multinest/nld_norm_0_.txt
2020-08-14 15:48:38,698 - ompy.normalizer_nld - INFO - Multinest results:
┌─────────────┬───────────────┬─────────────────┬──────────────┐
│ A           │ α [MeV⁻¹]     │ T [MeV]         │ Eshift [MeV] │
╞═════════════╪═══════════════╪═════════════════╪══════════════╡
│ 1.56 ± 0.12 │ 1.335

---
Note:

*We have not found a clear way of redirecting the complete multinest output to the notebook. Look in the terminal and be careful if multinest indicates that you converge towards the edge of a prior:*
```
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  400
 dimensionality =    4
 *****************************************************

 MultiNest Warning!
 Parameter            1  of mode            1  is converging towards the edge of the prior.

```

In [30]:
nldnorm.plot();

<IPython.core.display.Javascript object>

Observe that you might get *strange* results, i.e. unexpected results here, as you use the (potentially erroneous determined) uncertainties of `nld_mean` in the normalization, instead of the proper normalization below.

##### $\gamma$-SF Normalization

In [31]:
normlog = om.introspection.get_logger('normalizer_gsf', 'INFO')
gsfnorm = om.NormalizerGSF(normalizer_nld=nldnorm, gsf=extractor.gsf[0],
                           regenerate=True)

# to be use for gsf normalization
norm_pars.Gg = [113., 13.]  #meV

gsfnorm.norm_pars = norm_pars
gsfnorm.model_high.Efit = [4.5, 6.]

In [32]:
gsfnorm.normalize()
gsfnorm.plot()

2020-08-14 15:48:38,832 - ompy.normalizer_gsf - INFO - Normalizing #0
2020-08-14 15:48:38,837 - ompy.normalizer_gsf - INFO - Saving to saved_run/normalizers/NormalizerGSF.pkl


<IPython.core.display.Javascript object>

(<Figure size 448x336 with 1 Axes>,
 <AxesSubplot:xlabel='$\\gamma$-ray energy $E_\\gamma$~[MeV]', ylabel='$\\gamma$-SF f($E_\\gamma$)~[MeV$^{-3}$]'>)

It's often instructive to plot the extrapolation of the $\gamma$SF; with the interactive code below, we can check the influence of choosing different fit regions. The latest choice is kept for the simultaneous normalization below.

In [33]:
gsfnorm.plot_interactive()

2020-08-14 15:48:38,926 - ompy.normalizer_gsf - INFO - Normalizing #0
2020-08-14 15:48:38,929 - ompy.normalizer_gsf - INFO - Saving to saved_run/normalizers/NormalizerGSF.pkl


<IPython.core.display.Javascript object>

interactive(children=(SelectionRangeSlider(description='Elow', index=(2, 7), options=((1.1, 1.1), (1.3, 1.3), …

#### 2. Simultaneous:
We now propose to normalize the NLD and $\gamma$SF simultaneously instead. This way, we are guaranteed to get matching combinations of the normalization parameters $A$, $B$ and $\alpha$ for a given ensemble member.

In [ ]:
normlog = om.introspection.get_logger('normalizer_simultan', 'INFO')
simnorm = om.NormalizerSimultan(normalizer_nld=nldnorm, normalizer_gsf=gsfnorm,
                               regenerate=True)

# running faster than the default 400 given less precise answers
# For propper calc. you may rather increase this to > 400.
simnorm.multinest_kwargs["n_live_points"] = 300

simnorm.normalize(gsf=extractor.gsf[0], nld=extractor.nld[0])

2020-08-14 15:48:40,143 - ompy.normalizer_nld - INFO - DE results:
┌────────────────────┬────────────────────┬────────────────────┬──────────────────────┐
│ A                  │ α [MeV⁻¹]          │ T [MeV]            │ Eshift [MeV]         │
╞════════════════════╪════════════════════╪════════════════════╪══════════════════════╡
│ 1.3041657754481422 │ 1.3047666835840441 │ 0.5397053066859719 │ 0.033943533839594545 │
└────────────────────┴────────────────────┴────────────────────┴──────────────────────┘
2020-08-14 15:48:40,186 - ompy.normalizer_gsf - INFO - Normalizing #0
2020-08-14 15:48:40,190 - ompy.normalizer_simultan - INFO - DE results/initial guess:
┌────────────────────┬────────────────────┬────────────────────┬──────────────────────┬────────────────────┐
│ A                  │ α [MeV⁻¹]          │ T [MeV]            │ Eshift [MeV]         │ B                  │
╞════════════════════╪════════════════════╪════════════════════╪══════════════════════╪════════════════════╡
│ 1.304165

In [ ]:
simnorm.plot();

### 3. Normalization the whole ensemble
We have now also developed the tool-set to normalize each member of the extractor ensemble separately. This should provide a statistically more sound and robust normalization. 

Why so? As the $\chi^2$ error function is degenerate, a good minimizer should return sets of (nld, gsf) that need to be normalized with different coefficients $A$, $B$ and $\alpha$. Thus we should normalize each of these sets independently, and build up an uncertainty band only after the normalization. (Instead of the *traditional* approach of normalizing the mean of the sets)

Again, you decide whether you normalize sequentially, or, as we **recommend**, to normalize simultaneously.

Note that this will this may take several minutes!

#### Sequential normalization

In [ ]:
normlog = om.introspection.get_logger('ensembleNormalizer', 'INFO')
ensemblenorm_seq = om.EnsembleNormalizer(extractor=extractor, normalizer_nld=nldnorm,
                                         normalizer_gsf=gsfnorm, regenerate=True)
ensemblenorm_seq.normalize()

In [ ]:
ensemblenorm_seq.plot();

#### Simultaneous normalization

In [ ]:
ensemblenorm_sim = om.EnsembleNormalizer(extractor=extractor, normalizer_simultan=simnorm,
                                         regenerate=True)

ensemblenorm_sim.normalize()

In [ ]:
ensemblenorm_sim.plot();

Does this particular set of random sample look very strange (by "accident" i.e. a bad seed?)  
Try to look at some others, like here. If the problem persists, you should look at earlier stages of the analysis.

In [ ]:
ensemblenorm_sim.plot(n_plot=5, random_state=np.random.default_rng(65546645));

You may want to export/share the results in a plain text format. There is also a hook for this:

In [ ]:
# normlog = om.introspection.get_logger('ensembleNormalizer', 'DEBUG')
ensemblenorm_sim.save_results_txt(path="test")